In [1]:
import pandas as pd
import geopandas as gpd
import folium
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import os

In [7]:
csv_file = "nyc_stats_csv/do_zone_stats.csv"  # change to do_zone_stats.csv if needed
df = pd.read_csv(csv_file)

# Choose which column to visualize
metric = "med_tip_per_hour"  # or "med_tip_per_hour"
title = "Dropoff Zones – Median Tip per Hour ($)"

shapefile_path = "taxi_zones.zip"  # your local shapefile
gdf = gpd.read_file(shapefile_path)

gdf = gdf.to_crs("EPSG:4326")

gdf['LocationID'] = gdf['LocationID'].astype(int)
gdf = gdf.set_index('LocationID')

gdf = gdf[gdf.index.isin(df['DOLocationID'].unique())]



values = df[metric]
vmin = values.min()
vmax = values.max()
norm = mcolors.Normalize(vmin=vmin, vmax=vmax)
cmap = plt.get_cmap("YlOrRd")  # Yellow → Red


color_dict = {}
for _, row in df.iterrows():
    loc_id = int(row['DOLocationID'])
    val = row[metric]
    rgba = cmap(norm(val))
    color_dict[loc_id] = mcolors.to_hex(rgba)


m = folium.Map(location=[40.7128, -74.0060], zoom_start=11, tiles="cartodbpositron")


for loc_id, geom in gdf.geometry.items():
    color = color_dict.get(loc_id, "#cccccc")  # fallback gray
    folium.GeoJson(
        geom,
        style_function=lambda feature, c=color: {
            'fillColor': c,
            'color': 'black',
            'weight': 0.5,
            'fillOpacity': 0.7,
        }
    ).add_to(m)

# Add legend
from branca.colormap import linear
colormap = linear.YlOrRd_09.scale(vmin, vmax)
colormap.caption = title
colormap.add_to(m)


output_file = os.path.join("nyc_stats_csv", f"{metric}_map.html")
m.save(output_file)
print(f"Map saved: {output_file}")

Map saved: nyc_stats_csv/med_tip_per_hour_map.html
